# 파이썬으로 linear regression 구현하기

Andrew Ng 교수님의 Coursera 강의 assignment1
https://www.coursera.org/learn/machine-learning/home/info

Python 구현 코드는 John Wittenauer 를 기반으로 하였다. 
http://www.johnwittenauer.net/machine-learning-exercises-in-python-part-1/

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# 데이터를 불러온다.

path = '/data/'